In [97]:
import numpy as np
import glob
import os
import cv2
import time

def search_all_files_return_by_time_reversed(regex):
    return sorted(glob.glob(regex), key=lambda x: os.path.getmtime(x))

def get_points(path, size, black_dot, verbose = False):
    objp = np.zeros((size[0] * size[1], 3), np.float32)
    for r in range(0, size[1]):
        y = size[1] - r;
        for c in range(0, size[0]):
            x = size[0] - c;
            if r % 2 == 0:
                objp[c + r * size[0]] = [x * 2 + 1, y, 0]
            else:
                objp[c + r * size[0]] = [x * 2, y, 0]


    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = search_all_files_return_by_time_reversed(os.path.join(path, "*.jpg"))
    # Step through the list and search for chessboard corners
    output_path = os.path.join(path, "corners")
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    for idx, fname in enumerate(images):
        if verbose:
            print("read in:", fname)
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        shape = gray.shape

        #  findCirclesGrid only detect Dark Color, so revert color here.
        if not black_dot:
            full_white = np.ones(gray.shape, dtype=np.uint8) * 255;
            gray = np.array(full_white - gray);

        params = cv2.SimpleBlobDetector_Params()
        params.maxArea = 10e4
        params.minArea = 10
        params.minDistBetweenBlobs = 5
        blobDetector = cv2.SimpleBlobDetector_create(params)
        # Find the circle grid centers
        ret, corners = cv2.findCirclesGrid(gray, size, cv2.CALIB_CB_ASYMMETRIC_GRID, blobDetector, None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, size, corners, ret)
            write_name = 'corners_found'+str(idx)+'.jpg'
            cv2.imwrite(os.path.join(output_path, write_name), img)

        else: 
            print("no circle found.")

    imgpoints = np.squeeze(imgpoints)
    return images, objpoints, imgpoints, shape

def undistort_image(path, images, matrix, dist):
    output_path = os.path.join(path, "undistorted")
    if not os.path.exists(output_path):
        os.mkdir(output_path)

    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        img = cv2.undistort(img, matrix, dist, None, None)
        write_name = str(idx)+'.jpg'
        cv2.imwrite(os.path.join(output_path, write_name), img)

def K_to_homogenous_matrix(K):
    K = np.vstack((K, [0, 0, 0]));
    K = np.hstack((K, [[0], [0], [0], [1]]));
    return np.matrix(K)

def Rt_to_homogeneous_matrix(R, t, Rodrigues = True):
    if Rodrigues:
        rvec, _= cv2.Rodrigues(R, None)
    else:
        rvec = R
    transform = np.hstack((rvec, t))
    transform = np.vstack((transform, [0, 0, 0, 1]));
    return np.matrix(transform)


# calculate the transform from L to R
def calculate_W_L_to_R(K_l, Rt_l, K_r, Rt_r):
    W = K_r * Rt_r * Rt_l.I * K_l.I
    return np.matrix(W)


# Thermal images
print("Running thermal images calibration...")
t_images, t_obj_pos, t_img_pos, shape = get_points("thermal-img", (3, 9), False)
ret, thermal_K, t_dist, thermal_rvecs, thermal_tvecs = cv2.calibrateCamera(t_obj_pos, t_img_pos, shape, None, None)
undistort_image("thermal-img", t_images, thermal_K, t_dist)
print("Done!")


Rt_to_use = 0

thermal_transform = Rt_to_homogeneous_matrix(thermal_rvecs[Rt_to_use], thermal_tvecs[Rt_to_use])
print(thermal_transform)

thermal_K = K_to_homogenous_matrix(thermal_K)
print(thermal_K)

# Normal images
print("Running normal images calibration...")
images, obj_pos, img_pos, shape = get_points("normal-img", (3, 9), True)
ret, normal_K, dist, normal_rvecs, normal_tvecs = cv2.calibrateCamera(obj_pos, img_pos, shape, None, None)
undistort_image("normal-img", images, normal_K, dist)
print("Done!")

normal_transform = Rt_to_homogeneous_matrix(normal_rvecs[Rt_to_use], normal_tvecs[Rt_to_use])
print(normal_transform)

normal_K = K_to_homogenous_matrix(normal_K)
print(normal_K)

W = calculate_W_L_to_R(normal_K, normal_transform, thermal_K, thermal_transform);
print(W)

z = np.array(normal_transform)[2][3]
normal = cv2.imread("2.jpg")
thermal = cv2.imread("1.jpg")
height, width, _ = normal.shape

depth_img = cv2.imread("smvs-B2.jpg", cv2.IMREAD_GRAYSCALE)
dp_height, dp_width = depth_img.shape

depth_img = cv2.resize(depth_img, (width, height))
depth_img[depth_img == 0] = 0

#  z_R * [u_R, v_R, 1, 1/z_R]^T = W * z_L * [u_L, v_L, 1, 1/z_L]^T
normal_image_pos = np.zeros((4, width * height), np.float32)
normal_image_pos[0,:] = np.tile(np.arange(width),height)
normal_image_pos[1,:] = np.repeat(np.arange(height),width)
normal_image_pos[2,:] = 1
normal_image_pos[3,:] = 1 / depth_img.reshape(width * height)
thermal_test_img_pos = W * normal_image_pos

thermal_test_img_pos = thermal_test_img_pos / thermal_test_img_pos[2]

x_map = thermal_test_img_pos[0].reshape(height, width).astype(np.float32)
y_map = thermal_test_img_pos[1].reshape(height,width).astype(np.float32)

thermal_mapped = cv2.remap(thermal, x_map, y_map , cv2.INTER_LINEAR)
merged = cv2.addWeighted(normal, 0.5, thermal_mapped, 0.8, 0.2)
merged = cv2.resize(merged, (dp_width, dp_height))
cv2.imwrite("merged.jpg", merged)

Running thermal images calibration...
Done!
[[ 9.36093598e-01  6.67484254e-03  3.51687678e-01 -4.13045532e+00]
 [ 4.22212519e-02  9.90459143e-01 -1.31179468e-01 -5.27719205e+00]
 [-3.49207878e-01  1.37644954e-01  9.26880642e-01  4.30102301e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[1.18267651e+03 0.00000000e+00 3.22495403e+02 0.00000000e+00]
 [0.00000000e+00 1.18267651e+03 2.86178234e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Running normal images calibration...
no circle found.
no circle found.
no circle found.
Done!
[[ 9.47812162e-01 -6.62762980e-03  3.18760382e-01 -1.00154404e+01]
 [ 4.46052828e-02  9.92707661e-01 -1.11990488e-01 -4.85856122e+00]
 [-3.15693641e-01  1.20364343e-01  9.41196021e-01  4.19320144e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[2.69741324e+03 0.00000000e+00 1.23795509e+03 0.00000000e+00]
 [0.0000

<ipython-input-97-df3b671f3f95>:150: RuntimeWarning: divide by zero encountered in true_divide
  normal_image_pos[3,:] = 1 / depth_img.reshape(width * height)
<ipython-input-97-df3b671f3f95>:153: RuntimeWarning: invalid value encountered in true_divide
  thermal_test_img_pos = thermal_test_img_pos / thermal_test_img_pos[2]


True